# IBM Watson Library

Full details and documentation on the IBM Watson library, check it ou here: https://pypi.org/project/ibm-watson/

In [1]:
#  pip install --upgrade "ibm-watson>=5.0.0"

In [2]:
from ibm_watson import VisualRecognitionV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

import json

# Visual Recognition

### Authenticate and Set Service URL

In [4]:
authenticator = IAMAuthenticator('3n45UECd9WzYQCkVVxiqxigJqM-o6diiz-xF75C5FR-X')
authenticator

In [5]:
vis_recog = VisualRecognitionV3(
    version = '2018-03-19',
    authenticator = authenticator
)
vis_recog

In [6]:
vis_recog.set_service_url('https://api.eu-de.visual-recognition.watson.cloud.ibm.com/instances/3b667de7-e90a-4355-805c-52bd7571d94d')

### Set image

In [7]:
img_url = 'https://thefunnybeaver.com/wp-content/uploads/2017/07/angry-otter.jpg'

### Call service

In [8]:
classes_result = vis_recog.classify(url = img_url).get_result()

In [9]:
classes_result

{'images': [{'classifiers': [{'classifier_id': 'default',
     'name': 'default',
     'classes': [{'class': 'sea otter',
       'score': 0.914,
       'type_hierarchy': '/animal/mammal/carnivore/sea otter'},
      {'class': 'carnivore', 'score': 0.967},
      {'class': 'mammal', 'score': 0.967},
      {'class': 'animal', 'score': 0.967},
      {'class': 'otter',
       'score': 0.582,
       'type_hierarchy': '/animal/mammal/carnivore/otter'},
      {'class': 'black color', 'score': 0.729},
      {'class': 'chocolate color', 'score': 0.708}]}],
   'source_url': 'https://thefunnybeaver.com/wp-content/uploads/2017/07/angry-otter.jpg',
   'resolved_url': 'https://thefunnybeaver.com/wp-content/uploads/2017/07/angry-otter.jpg'}],
 'images_processed': 1,
 'custom_classes': 0}

In [11]:
print(json.dumps(classes_result, indent=2))

{
  "images": [
    {
      "classifiers": [
        {
          "classifier_id": "default",
          "name": "default",
          "classes": [
            {
              "class": "sea otter",
              "score": 0.914,
              "type_hierarchy": "/animal/mammal/carnivore/sea otter"
            },
            {
              "class": "carnivore",
              "score": 0.967
            },
            {
              "class": "mammal",
              "score": 0.967
            },
            {
              "class": "animal",
              "score": 0.967
            },
            {
              "class": "otter",
              "score": 0.582,
              "type_hierarchy": "/animal/mammal/carnivore/otter"
            },
            {
              "class": "black color",
              "score": 0.729
            },
            {
              "class": "chocolate color",
              "score": 0.708
            }
          ]
        }
      ],
      "source_url": "https://thef

### Quick function

In [12]:
def nice_classify(url):
    classes_result = vis_recog.classify(url = url).get_result()
    print(json.dumps(classes_result, indent=2))

## Built-In Classifiers

https://cloud.ibm.com/apidocs/visual-recognition/visual-recognition-v3?code=python#classify <br>
Check for explicit?!?!

In [13]:
nice_classify('https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fi.ytimg.com%2Fvi%2FqRPC6vOrIzE%2Fmaxresdefault.jpg&f=1&nofb=1')

{
  "images": [
    {
      "classifiers": [
        {
          "classifier_id": "default",
          "name": "default",
          "classes": [
            {
              "class": "hotdog",
              "score": 0.98,
              "type_hierarchy": "/food/nutrition/dish/sandwich/hotdog"
            },
            {
              "class": "sandwich",
              "score": 0.98
            },
            {
              "class": "dish",
              "score": 0.98
            },
            {
              "class": "nutrition",
              "score": 0.98
            },
            {
              "class": "food",
              "score": 0.98
            },
            {
              "class": "food product",
              "score": 0.799
            },
            {
              "class": "lemon yellow color",
              "score": 0.965
            }
          ]
        }
      ],
      "source_url": "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fi.ytimg.com%2Fvi%2FqR

In [16]:
def nice_classify(url, classifier = ['default']):
    classes_result = vis_recog.classify(
        url = url,
        classifier_ids = classifier
    ).get_result()
    print(json.dumps(classes_result, indent=2))

In [17]:
nice_classify('https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fi.ytimg.com%2Fvi%2FqRPC6vOrIzE%2Fmaxresdefault.jpg&f=1&nofb=1', classifier = ['default', 'food', 'explicit'])

{
  "images": [
    {
      "classifiers": [
        {
          "classifier_id": "default",
          "name": "default",
          "classes": [
            {
              "class": "hotdog",
              "score": 0.98,
              "type_hierarchy": "/food/nutrition/dish/sandwich/hotdog"
            },
            {
              "class": "sandwich",
              "score": 0.98
            },
            {
              "class": "dish",
              "score": 0.98
            },
            {
              "class": "nutrition",
              "score": 0.98
            },
            {
              "class": "food",
              "score": 0.98
            },
            {
              "class": "food product",
              "score": 0.799
            },
            {
              "class": "lemon yellow color",
              "score": 0.965
            }
          ]
        },
        {
          "classifier_id": "food",
          "name": "food",
          "classes": [
            {
  

## Custom Classifier

Full details here: https://cloud.ibm.com/apidocs/visual-recognition/visual-recognition-v3?code=python#createclassifier

### Create

In [18]:
with open('./Mantas.zip', 'rb') as mantas, open('./Stingrays.zip', 'rb') as stingrays, open('./Otters.zip', 'rb') as otters:
    model = vis_recog.create_classifier(
        name = 'Batoidea',
        positive_examples = {'manta_ray': mantas, 'stingray': stingrays},
        negative_examples = otters
    ).get_result()

ERROR:root:{'code': 400, 'error_id': 'input_error', 'description': 'Cannot execute learning task.  : this plan instance can have only 2 custom classifier(s), and 3 already exist.'}
Traceback (most recent call last):
  File "/Users/chungleu@uk.ibm.com/opt/anaconda3/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 225, in send
    response.status_code, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: {'code': 400, 'error_id': 'input_error', 'description': 'Cannot execute learning task.  : this plan instance can have only 2 custom classifier(s), and 3 already exist.'}, Code: 400 , X-global-transaction-id: bd20c2ce-2961-41df-aaae-1e019ee7245f


ApiException: Error: {'code': 400, 'error_id': 'input_error', 'description': 'Cannot execute learning task.  : this plan instance can have only 2 custom classifier(s), and 3 already exist.'}, Code: 400 , X-global-transaction-id: bd20c2ce-2961-41df-aaae-1e019ee7245f

In [ ]:
### List 

In [19]:
print(json.dumps(model, indent=2))

NameError: name 'model' is not defined

There are also APIs that support:
- retrieving a list of available classifiers
- retrieving classifier details
- updating classifieres
- deleting classifiers

### Use

In [20]:
nice_classify('https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fmedia.buzzle.com%2Fmedia%2Fimages-en%2Fgallery%2Ffish%2F1200-55844925-stingray-grand-cayman-island.jpg&f=1&nofb=1', ['default', 'Batoidea_550385256'])

{
  "images": [
    {
      "classifiers": [
        {
          "classifier_id": "default",
          "name": "default",
          "classes": [
            {
              "class": "electric ray",
              "score": 0.628,
              "type_hierarchy": "/animal/aquatic vertebrate/fish/ray/electric ray"
            },
            {
              "class": "ray",
              "score": 0.841
            },
            {
              "class": "fish",
              "score": 0.898
            },
            {
              "class": "aquatic vertebrate",
              "score": 0.9
            },
            {
              "class": "animal",
              "score": 0.93
            },
            {
              "class": "angel shark",
              "score": 0.536,
              "type_hierarchy": "/animal/aquatic vertebrate/fish/shark/angel shark"
            },
            {
              "class": "shark",
              "score": 0.561
            },
            {
              "class"

# Text to Speech

In [ ]:
from ibm_watson import TextToSpeechV1

In [ ]:
TTSauthenticator = IAMAuthenticator('jSnMxbNInJ2grxO1hUifi7JQ7Ivfc4Qccji0fXC08cJD')
text_to_speech = TextToSpeechV1(
    authenticator = TTSauthenticator
)

text_to_speech.set_service_url('https://gateway-lon.watsonplatform.net/text-to-speech/api')

## List Voices

In [ ]:
voices = text_to_speech.list_voices().get_result()

In [ ]:
print(json.dumps(voices, indent=2))

## Synthesise Voice

In [ ]:
synthesis = text_to_speech.synthesize(
    text = 'Buenos dias amigos. Como estas?',
    voice = 'es-ES_EnriqueV3Voice',
    accept='audio/wav'
).get_result()

In [ ]:
synthesis

In [ ]:
with open('Test.wav', 'wb') as audio_test:
    audio_test.write(synthesis.content)